In [32]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
schoolmistress_story_paragraphs = open('schoolmistress_paragraphs.txt', 'r').readlines()
el_story_paragraphs = open('expensivelessons_paragraphs.txt', 'r').readlines()

In [3]:
def parse_highlights(filename, story_paragraphs):
  highlight_file = open(f'./highlighting_logs/{filename}.log', 'r')
  cleaned_file = open(f'./backend/highlights/{filename.split("_")[0]}_{"schoolmistress" if "schoolmistress" in filename else "expensivelessons"}_cleaned.log', 'w')
  lines = highlight_file.readlines()
  highlights = [None] * len(story_paragraphs)
  for line in lines:
    start = line.find('Highlight:')
    if start != -1:
      highlight = json.loads(str(line[start + len('Highlight: '):].replace("'", '"')), )
      par = highlight.pop(-1)['paragraph']
      highlights[par] = json.dumps(highlight)
  for i in range(len(highlights)):
    if highlights[i] != None:
      cleaned_file.write(str(highlights[i]) + '\n')
    else:
      cleaned_file.write('[]\n')
  cleaned_file.close()
  

In [4]:
HIGHLIGHT_DIR = "./highlighting_logs/"
survey_df = pd.DataFrame()
for filename in os.listdir(HIGHLIGHT_DIR):
    f = os.path.join(HIGHLIGHT_DIR,filename)
    if os.path.isfile(f):
        filename = f.replace(HIGHLIGHT_DIR, '').replace('.log', '')
        if 'schoolmistress' in filename:
          parse_highlights(filename, schoolmistress_story_paragraphs)
        else:
          parse_highlights(filename, el_story_paragraphs)



In [60]:
def create_grid_of_coordinates_in_rectangle(top_left, bottom_right, width, height):
    x1, y1 = top_left
    x2, y2 = bottom_right
    x_vals = np.linspace(x1, x2, width, dtype=np.int64)
    y_vals = np.linspace(y1, y2, height, dtype=np.int64)
    x_grid, y_grid = np.meshgrid(x_vals, y_vals)
    return x_grid, y_grid

In [33]:
def check_if_coord_in_rectangle(coord, top_left, bottom_right):
    x, y = coord
    x1, y1 = top_left
    x2, y2 = bottom_right
    return x1 <= x <= x2 and y1 <= y <= y2

In [70]:
x_grid, y_grid = create_grid_of_coordinates_in_rectangle([0, 0], (1440, 900), 200, 100)

In [35]:
# print x_grid to file as json
with open('x_grid.txt', 'w') as f:
    f.write(json.dumps(x_grid.tolist()))

In [36]:
#print y_grid to file as json
with open('y_grid.txt', 'w') as f:
    f.write(json.dumps(y_grid.tolist()))

In [129]:
#get top left and bottom right from backend/logs/rose2.txt
with(open('backend/logs/schoolmistress_sentence_coords.txt', 'r')) as f:
    lines = f.readlines()

top_lefts_sm = []
bottom_rights_sm = []
sent_counts_sm = []
for line in lines:
    line_dict = json.loads(line.replace("'", '"'))
    if 'count' in line_dict.keys():
        sent_counts_sm.append(line_dict['count'])
    else:
        top_lefts_sm.append(line_dict['top_left'])
        bottom_rights_sm.append(line_dict['bottom_right'])
    

In [134]:
#get top left and bottom right from backend/logs/rose2.txt
with(open('backend/logs/expensivelessons_sentence_coords.txt', 'r')) as f:
    lines = f.readlines()

top_lefts_el = []
bottom_rights_el = []
sent_counts_el = []
for line in lines:
    line_dict = json.loads(line.replace("'", '"'))
    if 'count' in line_dict.keys():
        sent_counts_el.append(line_dict['count'])
    else:
        top_lefts_el.append(line_dict['top_left'])
        bottom_rights_el.append(line_dict['bottom_right'])

In [126]:
participant_df = pd.read_csv('backend/dwell_times/id2_schoolmistress.csv')
len(participant_df)

203

In [135]:
participant_df2 = pd.read_csv('backend/dwell_times/id2_el.csv')

In [147]:
def generate_coordinates(story, sent_counts, top_lefts, bottom_rights, x_grid, y_grid, story_len):
    total_count=0
    for page in range(len(sent_counts)):
        coord_dicts = []
        
        for coords in zip(x_grid.flatten(), y_grid.flatten()):
            coord_dict = {}
            coord_dict['x'] = coords[0]
            coord_dict['y'] = coords[1]
            for i in range(total_count, total_count + sent_counts[page]):
                if check_if_coord_in_rectangle(coords, top_lefts[i], bottom_rights[i]):
                    coord_dict['sentence'] = i
                    # coord_dict['dwell_time'] = int(participant_df.iloc[i]['IA_DWELL_TIME_SMOOTHED'] / participant_df.iloc[i]['word'])
                    coord_dicts.append(coord_dict)
        total_count=total_count+sent_counts[page]
        gaze_durations = pd.DataFrame.from_dict(coord_dicts)
        gaze_durations.to_csv(f'backend/coordinates/{story}{str(page)}.csv', index=False, header=False)

In [148]:
generate_coordinates('schoolmistress', sent_counts_sm, top_lefts_sm, bottom_rights_sm, x_grid, y_grid, len(participant_df))

In [149]:
generate_coordinates('expensivelessons', sent_counts_el, top_lefts_el, bottom_rights_el, x_grid, y_grid, len(participant_df2))

In [163]:
# read files in backend/dwell_times
# for each file, read in the dwell times
# for each page, read in the coordinates
# for each coordinate, add the dwell time to the coordinate
# write the coordinates to a file
dwell_time_dir = 'backend/dwell_times/'
for filename in os.listdir(dwell_time_dir):
    f = os.path.join(dwell_time_dir, filename)
    if os.path.isfile(f):
        filename = f.replace(dwell_time_dir, '').replace('.csv', '')
        if 'schoolmistress' in filename:
            story = 'schoolmistress'
        else:
            story = 'expensivelessons'
        participant_df = pd.read_csv(f)
        sent_counts = sent_counts_sm if story == 'schoolmistress' else sent_counts_el
        total_count=0
        for page in range(len(sent_counts)):
            coord_df = pd.read_csv(f'backend/coordinates/{story}{str(page)}.csv', header=None, dtype=np.int64)
            coord_df.columns = ['x', 'y', 'sentence']
            for i in range(total_count, total_count + sent_counts[page]):
                if i < len(participant_df):
                    coord_df.loc[coord_df['sentence'] == i, 'dwell_time'] = int(participant_df.iloc[i]['IA_DWELL_TIME_SMOOTHED'] / participant_df.iloc[i]['word'])
                else:
                    coord_df.loc[coord_df['sentence'] == i, 'dwell_time'] = int(participant_df.iloc[-1]['IA_DWELL_TIME_SMOOTHED'] / participant_df.iloc[-1]['word'])
            total_count=total_count+sent_counts[page]
            coord_df.drop(columns=['sentence'], inplace=True)
            coord_df['dwell_time'] = coord_df['dwell_time'].astype(np.int64)
            coord_df.to_csv(f'backend/heatmap/{filename}-{str(page)}.csv', index=False, header=False)

In [159]:
participant_df.iloc[0]['IA_DWELL_TIME_SMOOTHED']

346.0

In [ ]:
# sentence object
class Sentence:
    def __init__(self, sentence, paragraph, sentence_index, paragraph_index, story_index, story_name):
        self.sentence = sentence
        self.paragraph = paragraph
        self.sentence_index = sentence_index
        self.paragraph_index = paragraph_index
        self.story_index = story_index
        self.story_name = story_name
        self.highlighted = False
        self.highlighted_by = []
        self.highlighted_by_index = []
        self.highlighted_by_story_index = []
        self.highlighted_by_story_name = []
        self.highlighted_by_sentence_index = []
        self.highlighted_by_paragraph_index = []
        self.highlighted_by_sentence = []
        self.highlighted_by_paragraph = []
        self.highlighted_by_sentence_index = []

In [28]:
# read lines in backend/stories/schoolmistress.txt
with(open('backend/stories/expensivelessons.txt', 'r')) as f:
    story_lines = f.readlines()

# split lines by punctuation
all_sentences = []
for line in story_lines:
    line = line.replace('?', '.').replace('!', '.').replace("....\"", ",\"").replace("....", ".").replace('”', '"').replace('“', '"').replace('’', "'").replace('‘', "'")
    sentences = line.split('.')
    # remove empty strings and quotes
    sentences = [sentence for sentence in sentences if sentence != '' and sentence != '"' and sentence != '"\n' and sentence != '\n']
    sentences = list(map(lambda sentence: "<span>"+sentence+"</span>", sentences))
    all_sentences.append(sentences)

In [29]:
with open('backend/stories/expensivelessons_modified.txt', 'w') as f:
    for sentences in all_sentences:
        for sentence in sentences:
            f.write(sentence+'.')
        f.write('\n')

In [43]:
obj = '{"top_left": [1, 2], "bottom_right": [2, 4]}'
print(json.loads(obj).get('top_left'))

[1, 2]


In [ ]:
# read rose2.txt